# Overview
1. Setup
1. Use Case Background
1. Hyperparameter (HP) Definition
1. Decision Tree Recap
1. Data Preparation
1. Description of HPs Decision trees (i.e. what the HPs do, how they affect the model, and tradeoffs) 
1. Build simple decision tree model with no tuning
1. Tuning Methods
    + Grid Search
    + Random Search
    + Bayesian Search
1. Tuning with Random Forests (Optional)
1. Appendix

# Setup
Perform the following steps:

__Mac Users:__
- Open terminal
    - Change directory: `cd Desktop`
    - Clone the repository: `git clone https://github.com/benattix/ml_tutorials.git`
    - Change directory: `cd ml_tutorials`
    - Create new conda environment with .yml file: `conda env create -f environment_mac.yml`
    - Activate new environment `source activate hptuning`
    - Launch a Jupyter notebook: `jupyter notebook`

__Windows Users:__
- Open git bash
    - Change directory: `cd Desktop`
    - Clone the repository: `git clone https://github.com/benattix/ml_tutorials.git`
- Open anaconda prompt
    - Change directory: `cd Desktop/ml_tutorials`
    - Create new conda environment with .yml file: `conda env create -f environment_windows.yml`
    - Activate new environment `conda activate hptuning`
    - Launch a Jupyter notebook: `jupyter notebook`

# Background
We have a data set of beers and certain characteristics about the beer. Our goal is to determine for a given beer, how bitter it will be. Some people really enjoy bitter beers while other people hate them. By predicting how bitter a beer will be we can help people better select beers that match their tastes.

The data set contains the following variables:
+ abv - Alcohol by volume
+ ibu - International Bitterness Units. This is a gauge of beer's bitterness
+ id - unique identifier of each beer
+ name - name of the beer
+ style - style of the beer
+ brewery_id - id of the brewery where the beer was made
+ ounces - number of fluid ounces that the beer is sold in

For our task, we will try to predict the `ibu`, since that is a measure of the bitterness.

The dataset comes from [Kaggle](https://www.kaggle.com/nickhould/craft-cans)

# Hyperparameter Definition
A hyperparameter is a parameter whose value is set before the learning process begins. Parameters, on the other hand, are calculated during training (ex. linear regression coefficients).

# Quick Decision Tree Recap

In Decision Trees, we start from the root of the tree. We compare the values of the root attribute with the record’s attribute. On the basis of comparison, we follow the branch corresponding to that value and jump to the next node.

<img src="images/decision_tree.jpg" width=500>

For more detail on how decision trees work, see [Decision Tree Algorithm, Explained](https://www.kdnuggets.com/2020/01/decision-tree-algorithm-explained.html)

## Read Data

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('data/beers.csv', index_col=0)
print(df.shape)
df.head(3)

(2410, 7)


,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0


We can see that there are some null values. Let's check for nulls across each column.

In [4]:
df.isna().sum()

abv             62
ibu           1005
id               0
name             0
style            5
brewery_id       0
ounces           0
dtype: int64

## Preprocessing

Since we are trying to predict `ibu`, we will drop any rows where it is null. We cannot measure our accuracy if we do not know the true value that we're trying to predict.

In [5]:
# drop rows with null 'ibu'
df = df.dropna(subset=['ibu'], axis=0)
df.sort_values(by='id', inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(1405, 7)

For the other columns with missing values, we will fill them. Since `abv` is continuous, we will fill with the median and since `style` is categorical, we will create a new category which denotes that we do not know the style.

In [6]:
df['abv'].fillna(df.abv.median(), inplace=True)
df['style'].fillna('NA', inplace=True)

In [7]:
# drop id columns
df.drop(['id', 'brewery_id'], axis=1, inplace=True)
df.head(3)

,abv,ibu,name,style,ounces
0,0.065,65.0,Dale's Pale Ale,American Pale Ale (APA),12.0
1,0.087,85.0,Gordon Ale (2009),American Double / Imperial IPA,12.0
2,0.080,35.0,Old Chub,Scottish Ale,12.0


Next, we need to handle our categorical variables (name and style). To do this, let's use sci-kit learn's CountVectorizer, which converts a collection of text to a matrix of token (aka word) counts. In other words, it will create a new column for each individual token, and count how many times each token appeared in each row.

In [14]:
vectorizer = CountVectorizer(min_df=5, binary=True)
vec_matrix = vectorizer.fit_transform(df['name'])
df_name = pd.DataFrame(vec_matrix.todense(), columns=['name_' + i for i in vectorizer.get_feature_names()])

vec2 = CountVectorizer(min_df=5, binary=True) 
vec2_matrix = vec2.fit_transform(df['style'])
df_style = pd.DataFrame(vec2_matrix.todense(), columns=['style_' + i for i in vec2.get_feature_names()])

In [15]:
df_final = df.drop(['name', 'style'], axis=1)\
             .join(df_name)\
             .join(df_style)

df_final.head()

,abv,ibu,ounces,name_2006,name_2008,name_2009,name_2010,name_2011,name_2012,name_2013,...,style_sweet,style_vegetable,style_vienna,style_warmer,style_wee,style_weissbier,style_wheat,style_white,style_winter,style_witbier
0,0.065,65.0,12.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.087,85.0,12.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.080,35.0,12.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.099,100.0,12.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.053,35.0,12.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create Training and Testing Sets

In [16]:
# split into train and test sets
X = df_final.drop(['ibu'], axis=1)
y = df_final['ibu']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

## Build Model Without Tuning

In [17]:
dt = DecisionTreeRegressor(random_state=3)
dt.fit(X_train, y_train)
print('R-squared - test set: %3.3f' % dt.score(X_test, y_test)) 

R-squared - test set: 0.637


# Specific Hyperparameters
## Decision Tree
Below are some of the hyperparameters that can be adjusted for a decision tree model. For more hyperparameters, see the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html).
+ `max_depth` : The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
+ `max_features` : The number of features to consider when looking for the best split:
    + If int, then consider max_features features at each split.
    + If float, then max_features is a fraction and int(max_features * n_features) features are considered at each split.
    + If “auto”, then max_features=n_features.
    + If “sqrt”, then max_features=sqrt(n_features).
    + If “log2”, then max_features=log2(n_features).
    + If None, then max_features=n_features.
+ `min_samples_leaf` : The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.
    + If int, then consider min_samples_leaf as the minimum number.
    + If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.
+ `min_impurity_decrease` : A node will be split if this split induces a decrease of the impurity greater than or equal to this value.

    The weighted impurity decrease equation is the following:

    __N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)__
    
    where N is the total number of samples, N_t is the number of samples at the current node, N_t_L is the number of samples in the left child, and N_t_R is the number of samples in the right child.

    N, N_t, N_t_R and N_t_L all refer to the weighted sum, if sample_weight is passed.

# Tuning Methods
1. [Grid Search](#grid)
1. [Random Search](#random)
1. [Bayesian Search](#bayes)

<a id="grid"></a>
## Grid Search
The first method of hyperparameter tuning that we will cover is grid search. The advantages of grid search is that, compared to manually searching the hyperparamter space, grid search allows you to search more space with less code
and it's more practical and exhaustive.

We will use sci-kit learn's implementation of grid search, [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). We pass the GridSearchCV object a model and the values of the hyperparamter space for it to try and it builds a new model for each hyperparamter combination. 

It's important when tuning hyperparamters to still use a hold out (i.e. test) set so you can understand how well your model generalizes to data it did not see during training.

<img src="images/grid_search.gif" width=500>

Souce: [SigOpt](https://sigopt.com/blog/common-problems-in-hyperparameter-optimization)

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth': list(range(2, 15)),
              'min_samples_leaf': list(range(1,15,2)),
              'min_impurity_decrease': [0.00005,0.0001,0.0005,0.001,0.005,0.01,0.1]
              } 

In [20]:
grid = GridSearchCV(dt, param_grid, cv=10, verbose=1)
grid.fit(X_train, y_train)

print('Best parameters:', grid.best_params_)
print('Best cross-validation R-squared: %3.3f' % grid.best_score_)
print('R-squared - test set: %3.3f' % grid.score(X_test, y_test)) 

Fitting 10 folds for each of 1911 candidates, totalling 19110 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best parameters: {'max_depth': 7, 'max_features': 'auto', 'min_impurity_decrease': 0.1, 'min_samples_leaf': 3}
Best cross-validation R-squared: 0.731
R-squared - test set: 0.737


[Parallel(n_jobs=1)]: Done 19110 out of 19110 | elapsed:  1.6min finished


<a id="random"></a>
## Random Search
When using grid search above, there was a very large hyperparameter space and it can be computationally expensive to try every single combination. 

With random search, we only try a subset of the combinations to avoid some unnecessary computation. Since we're only trying a subset, it's possible that the optimal hyperparameter combination will not be chosen and as a result, we could end up with slightly worse performance. 


We will use sci-kit learn's implementation of random search, [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). RandomizedSearchCV's `n_iter` parameter determines how many hyperparamter combinations we try. With the `n_iter` parameter, we can drastically reduce how much of the hyperparameter space that we explore. 

With RandomizedSearchCV, we can pass hyperparameters in 2 different ways:
1. With a dictionary of values just as we did above (for GridSearchCV)
1. With a statistical distribution. When we pass a distribution, then for each iteration the algorithm will randomly sample a value from that distrubition. Using distrubitions introduces more variation and allows for the possibility of finding an optimal hyperparameter in an area that you would not have chosen to guess.

<img src="images/random_search.gif" width=500>

Source: [SigOpt](https://sigopt.com/blog/common-problems-in-hyperparameter-optimization)

First, let's use the same predetermined values that we used for GridSearchCV.

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [22]:
randomsearch = RandomizedSearchCV(dt, 
                                  param_grid, 
                                  cv=10, 
                                  n_iter=40, 
                                  random_state=3,
                                  verbose=1)
randomsearch.fit(X_train, y_train)

print('Best parameters:', randomsearch.best_params_)
print('Best cross-validation R-squared: %3.3f' % randomsearch.best_score_)
print('R-squared - test set: %3.3f' % randomsearch.score(X_test, y_test)) 

Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best parameters: {'min_samples_leaf': 3, 'min_impurity_decrease': 0.0005, 'max_features': 'auto', 'max_depth': 8}
Best cross-validation R-squared: 0.727
R-squared - test set: 0.736


[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    2.0s finished


Now let's try RandomizedSearchCV using distributions instead of fixed values.

In [23]:
param_grid_dist = {'max_features': ['auto', 'sqrt', 'log2'],
                   'max_depth': stats.randint(1,15),
                   'min_samples_leaf': stats.randint(1,15),
                   'min_impurity_decrease': stats.uniform(1e-4, 1e-1)
                   } 

In [24]:
randomsearch2 = RandomizedSearchCV(dt, 
                                   param_grid_dist, 
                                   cv=10, 
                                   n_iter=40, 
                                   random_state=12)
randomsearch2.fit(X_train, y_train)

print('Best parameters:', randomsearch2.best_params_)
print('Best cross-validation R-squared: %3.3f' % randomsearch2.best_score_)
print('R-squared - test set: %3.3f' % randomsearch2.score(X_test, y_test))

Best parameters: {'max_depth': 9, 'max_features': 'auto', 'min_impurity_decrease': 0.07625399142613266, 'min_samples_leaf': 4}
Best cross-validation R-squared: 0.730
R-squared - test set: 0.738


In [25]:
randomsearch2.cv_results_['params']

[{'max_depth': 12,
  'max_features': 'log2',
  'min_impurity_decrease': 0.08743871524282919,
  'min_samples_leaf': 3},
 {'max_depth': 4,
  'max_features': 'auto',
  'min_impurity_decrease': 0.035147827600499254,
  'min_samples_leaf': 2},
 {'max_depth': 5,
  'max_features': 'sqrt',
  'min_impurity_decrease': 0.003442142762634459,
  'min_samples_leaf': 3},
 {'max_depth': 12,
  'max_features': 'log2',
  'min_impurity_decrease': 0.037383446106186775,
  'min_samples_leaf': 6},
 {'max_depth': 9,
  'max_features': 'auto',
  'min_impurity_decrease': 0.09452251360530423,
  'min_samples_leaf': 10},
 {'max_depth': 4,
  'max_features': 'log2',
  'min_impurity_decrease': 0.019087634421308352,
  'min_samples_leaf': 2},
 {'max_depth': 8,
  'max_features': 'auto',
  'min_impurity_decrease': 0.07691341540644224,
  'min_samples_leaf': 3},
 {'max_depth': 1,
  'max_features': 'auto',
  'min_impurity_decrease': 0.056414113751657155,
  'min_samples_leaf': 14},
 {'max_depth': 10,
  'max_features': 'sqrt',
  

As you can see, RandomizedSearchCV chooses values from the distrubutions we provided. This allows us to search hyperparameter values that we might not have thought to use. This can lead to either better or worse results than GridSearchCV, depending on randomness.

<a id="bayes"></a>
## Bayesian Search
Instead of randomly searching the hyperparameter space, we can search it more efficiently. With the previous 2 methods, we experimented with different hyperparameter values and these experiments were independent of each other. Since they were independent, they were not able to use the information from one experiment to improve the next experiment.

Bayesian search addresses this issue. With bayesian search, experiments are run sequentially and information from each of the previous runs are used to determine the next set of hyperparameters to try. Bayesian search uses concepts of exploration and exploitation. First, it explores the hyperparameter space by choosing some initial hyperparameter values. Then once it has some understanding of the hyperparameter space, it tries to "exploit" it by focusing on areas of the hyperparameter space that give the best model performance and trying slight variations of values that it already knows are good.



<img src="images/bayesian_search.gif" width=500>

Souce: [SigOpt](https://sigopt.com/blog/common-problems-in-hyperparameter-optimization)

Additional Resources:
+ https://github.com/fmfn/BayesianOptimization
+ https://github.com/hyperopt/hyperopt
+ https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV

In [26]:
from skopt import BayesSearchCV

In [27]:
grid = {
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': (2, 15),
        'min_samples_leaf': (2, 15),
        'min_impurity_decrease': (0.0, 0.5, 'uniform'),
        }

In [ ]:
bayes = BayesSearchCV(dt,
                      grid,
                      n_iter=40,
                      cv=10
                     )

bayes.fit(X_train, y_train)

print('The best parameters are:', bayes.best_params_)
print('The best model cross-validation R-Squared is %3.3f' % bayes.best_score_)
print('R-squared - test set: %3.3f' % bayes.score(X_test, y_test)) 

# Hyperparameter Tuning on Random Forest (Optional)
Now that we have shown how to perform hyperparameter tuning on a decision tree model, try to tune a random forest model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
param_grid_rf = {'max_features': ['auto', 'sqrt', 'log2'],
                 'max_depth': [],   # set the max depth values to test
                 'n_estimators': [] # set the n_estimators values to test
                 }

In [ ]:
grid = GridSearchCV(RandomForestRegressor(), param_grid_rf)

In [ ]:
param_grid_dist = {'max_features': ['auto', 'sqrt', 'log2'],
                   'max_depth': , # set a distribution here
                   'n_estimators': # set a distribution here
                   } 

In [ ]:
randomsearch3 = RandomizedSearchCV(RandomForestRegressor(), 
                                   param_grid_dist, 
                                   cv=10, 
                                   n_iter=40, 
                                   random_state=12)

In [ ]:
bayes_grid = {
              'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth': (), # set min and max values here
              'n_estimators'(): # set min and max values here
              }

In [ ]:
bayes = BayesSearchCV(RandomForestRegressor(),
                      bayes_grid,
                      n_iter=40,
                      cv=10
                     )

# Appendix
Many cloud platforms can perform automated hyperparameter tuning for you. By using the cloud for tuning, you can parallelize (train multiple models at once) and use more powerful machines.
+ AWS SageMaker
    + https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html
    + https://sagemaker.readthedocs.io/en/stable/tuner.html
+ Azure
    + https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
+ GCP
    + https://cloud.google.com/ai-platform/training/docs/using-hyperparameter-tuning